In [1]:
with open("/home/coder/low-power-cv-challenge-track3/data/places365/metadata/IO_places365.txt") as f:
    metadata = f.readlines()

In [2]:
pattern = '\/[a-z]\/([\w\/]+)\s+([12])'
import re
category_labels = {}
for item in metadata:
    match = re.search(pattern, item)
    if match:
        category = match.group(1)
        where = match.group(2)
        category_labels[category] = where
category_labels_renamed = {key.replace("/", "-"): value for key, value in category_labels.items()}

In [4]:
import os
import glob
label_counts = {'1': 0, '2': 0}
folders = ["/Users/krishnaiyer/low-power-cv-challenge-track3/data/places365/places365_standard/train"]
for folder in folders:
    for category, label in category_labels.items():
        # Construct the path to the category folder
        category_path = os.path.join(folder, category)
        
        # Count the number of .jpg files using glob
        jpg_files = glob.glob(os.path.join(category_path, "*.jpg"))
        
        # Update label count
        label_counts[label] += len(jpg_files)


### indoor-outdoor splits in training data

In [26]:
print(label_counts)
print("Total images: ", sum(label_counts.values()))
print("Percentage of label indoor: ", label_counts['1']/sum(label_counts.values()))
print("Percentage of label outdoor: ", label_counts['2']/sum(label_counts.values()))

{'1': 625754, '2': 809138}
Total images:  1434892
Percentage of label indoor:  0.4360983265639505
Percentage of label outdoor:  0.5639016734360496


### indoor-outdoor splits in val data

In [28]:
print(label_counts)
print("Total images: ", sum(label_counts.values()))
print("Percentage of label indoor: ", label_counts['1']/sum(label_counts.values()))
print("Percentage of label outdoor: ", label_counts['2']/sum(label_counts.values()))

{'1': 12800, '2': 16200}
Total images:  29000
Percentage of label indoor:  0.4413793103448276
Percentage of label outdoor:  0.5586206896551724


## data augmentation sample calculation

In [9]:
train_counts = {'1': 625754, '2': 809138}
train_total = train_counts['1'] + train_counts['2']
train_outdoor_aug = int(round(train_counts['2'] * 0.10))
train_final_outdoor = train_counts['2'] + train_outdoor_aug
train_indoor_extra = train_final_outdoor - train_counts['1']

# For validation:
val_counts = {'1': 12800, '2': 16200}
val_total = val_counts['1'] + val_counts['2']
val_outdoor_aug = int(round(val_counts['2'] * 0.10))
val_final_outdoor = val_counts['2'] + val_outdoor_aug
val_indoor_extra = val_final_outdoor - val_counts['1']

print("---- TRAIN DATA CALCULATIONS ----")
print(f"Indoor images: {train_counts['1']}")
print(f"Outdoor images: {train_counts['2']}")
print(f"Outdoor augmented images (10% extra): {train_outdoor_aug}")
print(f"Final outdoor count: {train_final_outdoor}")
print(f"Additional indoor images required: {train_indoor_extra}")
print(f"final train data count {train_counts['1'] + train_final_outdoor + train_indoor_extra}")


print("---- VAL DATA CALCULATIONS ----")
print(f"Indoor images: {val_counts['1']}")
print(f"Outdoor images: {val_counts['2']}")
print(f"Outdoor augmented images (10% extra): {val_outdoor_aug}")
print(f"Final outdoor count: {val_final_outdoor}")
print(f"Additional indoor images required: {val_indoor_extra}")
print(f"final val data count {val_counts['1'] + val_final_outdoor + val_indoor_extra}")


---- TRAIN DATA CALCULATIONS ----
Indoor images: 625754
Outdoor images: 809138
Outdoor augmented images (10% extra): 80914
Final outdoor count: 890052
Additional indoor images required: 264298
final train data count 1780104
---- VAL DATA CALCULATIONS ----
Indoor images: 12800
Outdoor images: 16200
Outdoor augmented images (10% extra): 1620
Final outdoor count: 17820
Additional indoor images required: 5020
final val data count 35640


In [5]:
import torchvision.transforms as transforms
from PIL import Image

def apply_color_jitter(image_path: str, brightness: float = 0.5, contrast: float = 0.5,
                        saturation: float = 0.5, hue: float = 0.1) -> Image.Image:
    """
    Applies ColorJitter transformation to an input image.
    
    Parameters:
        image_path (str): Path to the input image.
        brightness (float): Factor to adjust brightness.
        contrast (float): Factor to adjust contrast.
        saturation (float): Factor to adjust saturation.
        hue (float): Factor to adjust hue.

    Returns:
        Image.Image: Transformed image with jittered colors.
    """
    # Load image
    image = Image.open(image_path).convert("RGB")
    
    # Define the ColorJitter transformation
    transform = transforms.ColorJitter(
        brightness=brightness, 
        contrast=contrast, 
        saturation=saturation, 
        hue=hue
    )
    
    # Apply transformation
    jittered_image = transform(image)
    
    return jittered_image


In [6]:
import random
from tqdm import tqdm
IMAGE_EXTENSIONS = {'.jpg'}
def augment_images_in_folder(folder_path: str, label: str, total_label_original: int, total_extra_required: int, aug_params: dict):
    """
    Augment images in a given folder based on the label.
    For outdoor (label "2"), extra images = 10% of images in that folder.
    For indoor (label "1"), extra images are assigned proportionally to meet overall extra required.
    """
    # List image files (skip files that already have "_jitter" in their name)
    all_files = os.listdir(folder_path)
    image_files = [f for f in all_files if os.path.splitext(f)[1].lower() in IMAGE_EXTENSIONS and "_jitter" not in f]

    if not image_files:
        return

    # Determine number of augmented images to generate for this folder
    if label == "2":
        num_aug = int(round(len(image_files) * 0.10))
    elif label == "1":
        # Proportionally allocate based on the number of images in this folder
        num_aug = int(round((len(image_files) / total_label_original) * total_extra_required))
    else:
        # If label is not recognized, do nothing.
        return

    # If no augmentation needed, return.
    if num_aug <= 0:
        return

    # Randomly sample without replacement
    # If requested number exceeds available images, sample all (without repeating)
    sample_count = min(num_aug, len(image_files))
    selected_files = random.sample(image_files, sample_count)

    for filename in tqdm(selected_files,desc=f"Augmenting..."):
        file_path = os.path.join(folder_path, filename)
        try:
            # Apply jitter using provided function and augmentation parameters
            jittered_img = apply_color_jitter(file_path, **aug_params)
            # Construct new filename
            name, ext = os.path.splitext(filename)
            new_filename = f"{name}_jitter.jpg"
            output_path = os.path.join(folder_path, new_filename)
            jittered_img.save(output_path)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

In [7]:
def process_dataset(root_dir: str, total_indoor: int, indoor_extra_required: int):
    """
    Process a dataset (train or val) directory. Each subfolder is a category.
    Augment images based on indoor-outdoor label.
    """
    # Augmentation parameters
    indoor_aug_params = {
        "brightness": 0.6,
        "contrast": 0.6,
        "saturation": 0.6,
        "hue": 0.1
    }
    outdoor_aug_params = {
        "brightness": 0.5,
        "contrast": 0.5,
        "saturation": 0.5,
        "hue": 0.05
    }
    for category in os.listdir(root_dir):
        category_path = os.path.join(root_dir, category)
        if not os.path.isdir(category_path):
            continue

        # Get the label for the category from the mapping
        label = category_labels_renamed.get(category)
        if label is None:
            print(f"Label for category '{category}' not found. Skipping...")
            continue

        if label == "2":
            print(f"Processing outdoor category: {category}")
            augment_images_in_folder(category_path, label, total_label_original=0, total_extra_required=0, aug_params=outdoor_aug_params)
        elif label == "1":
            print(f"Processing indoor category: {category}")
            augment_images_in_folder(category_path, label, total_label_original=total_indoor, total_extra_required=indoor_extra_required, aug_params=indoor_aug_params)
        else:
            print(f"Unknown label '{label}' for category '{category}'. Skipping...")


In [2]:
import os

def delete_jitter_images(main_folder):
    """
    Recursively deletes all .jpg files ending with '_jitter' in the given main folder and its subfolders.
    """
    for root, _, files in os.walk(main_folder):
        for file in files:
            if file.endswith("_jitter.jpg"):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

delete_jitter_images("/home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val")

Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00013929_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00032166_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00020760_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00034009_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00012632_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00025988_jitter.jpg
Deleted: /home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val/topiary_garden/Places365_val_00012339_jitter.jpg
Deleted: /home/coder/low-power-cv-

In [11]:
train_dir = "/home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/train"
val_dir = "/home/coder/low-power-cv-challenge-track3/data/places365/places365_standard/val"

print("Starting augmentation for training data...")
process_dataset(train_dir, total_indoor=train_counts['1'], indoor_extra_required=train_indoor_extra)

print("\nStarting augmentation for validation data...")
process_dataset(val_dir, total_indoor=val_counts['1'], indoor_extra_required=val_indoor_extra)

Starting augmentation for training data...
Processing outdoor category: vineyard


Augmenting...: 100%|██████████| 500/500 [00:03<00:00, 133.65it/s]


Processing indoor category: server_room


Augmenting...: 100%|██████████| 2112/2112 [00:15<00:00, 139.90it/s]


Processing outdoor category: vegetable_garden


...

Augmenting...: 100%|██████████| 500/500 [00:03<00:00, 129.78it/s]


Processing indoor category: art_school


Augmenting...: 100%|██████████| 2112/2112 [00:15<00:00, 133.32it/s]


Processing indoor category: laundromat


Augmenting...:   7%|▋         | 156/2112 [00:01<00:13, 141.28it/s]

### Verifying sample counts

In [63]:
import os
import glob
label_counts = {'1': 0, '2': 0}
folders = ["/Users/krishnaiyer/low-power-cv-challenge-track3/data/places365/places365_standard/val"]
for folder in folders:
    for category, label in category_labels.items():
        # Construct the path to the category folder
        category_path = os.path.join(folder, category)
        
        # Count the number of .jpg files using glob
        jpg_files = glob.glob(os.path.join(category_path, "*.jpg"))
        
        # Update label count
        label_counts[label] += len(jpg_files)


### training data

In [62]:
print(label_counts)
print("Total images: ", sum(label_counts.values()))
print("Percentage of label indoor: ", label_counts['1']/sum(label_counts.values()))
print("Percentage of label outdoor: ", label_counts['2']/sum(label_counts.values()))

{'1': 890070, '2': 890052}
Total images:  1780122
Percentage of label indoor:  0.5000050558332518
Percentage of label outdoor:  0.4999949441667481


### val data

In [64]:
print(label_counts)
print("Total images: ", sum(label_counts.values()))
print("Percentage of label indoor: ", label_counts['1']/sum(label_counts.values()))
print("Percentage of label outdoor: ", label_counts['2']/sum(label_counts.values()))

{'1': 17792, '2': 17820}
Total images:  35612
Percentage of label indoor:  0.4996068740873863
Percentage of label outdoor:  0.5003931259126138
